In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# At the beginning of the notebook
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
from playwright.async_api import async_playwright

p = await async_playwright().__aenter__()
browser = await p.chromium.launch(headless=False)
page = await browser.new_page(viewport={'width': 1280, 'height': 1000})

In [ ]:
from boto3 import Session
from mypy_boto3_bedrock_runtime import BedrockRuntimeClient

session = Session(profile_name="default", region_name="us-west-2")
bedrock_runtime: BedrockRuntimeClient = session.client(
    service_name="bedrock-runtime",
)

s3 = session.client('s3')

In [ ]:
from webrock.browser import AsyncBrowserPage, PlaywrightBrowserPage
from webrock.tools import ClickTool, TypeTextTool, PressKeyTool
from webrock.core import WebrockAgentExecutor
from webrock.bedrock import BedrockModel, ClaudeSonnet_3_5_BedrockModel
from webrock.artifacts import S3ArtifactPublisher

import webrock

await page.goto("https://www.amazon.com")

model = ClaudeSonnet_3_5_BedrockModel(bedrock_runtime=bedrock_runtime)
tools = [ClickTool, TypeTextTool, PressKeyTool]

browser_page: AsyncBrowserPage = PlaywrightBrowserPage(page)

agent = WebrockAgentExecutor(model=model, browser_page=browser_page, tools=tools)

prompt = "Search for 'bicycles', hit enter and click on one of the products. Once you reach the bicycle product page, stop"
actions = await agent.ainvoke(prompt, max_sequences=8)

In [ ]:
import json
import pprint

for action in actions:
    print(action.model_dump_json(indent=4))
    print()

In [ ]:
await browser.close()